In [5]:
"""
Created on Tue Feb 12 23:34:44 2019
@author: tobia
"""
import time
import datetime
import urllib.request, json

import numpy as np

import pandas as pd
from datetime import datetime, timedelta
from oceanoobsbrasil.db import GetData

from dotenv import load_dotenv
import os
import requests


In [6]:
%load_ext autoreload
%autoreload 2
from oceanoobsbrasil.weather_stations.inmet import Inmet
from oceanoobsbrasil.buoys.pnboia import Pnboia

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
i = Inmet()

In [8]:
i.get(save_bd=True)

A110
A013
A021
A023
A253
A302
A309
A310


KeyboardInterrupt: 

In [36]:
start_date=(datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = (datetime.utcnow() + timedelta(days=0)).strftime('%Y-%m-%d')
api="https://apitempo.inmet.gov.br"

In [37]:
db = GetData()
stations = db.get(table='stations', institution=['=', 'inmet'])

In [38]:
stations

,id,name,lat,lon,data_type,institution,url
0,285,BRASILIA,-15.7893,-47.9258,meterological_station,inmet,A001
1,286,GOIANIA,-16.6428,-49.2202,meterological_station,inmet,A002
2,287,MORRINHOS,-17.7451,-49.1017,meterological_station,inmet,A003
3,288,PORANGATU,-13.3095,-49.1175,meterological_station,inmet,A005
4,289,PALMAS,-10.1907,-48.3018,meterological_station,inmet,A009
...,...,...,...,...,...,...,...
583,868,NOVA ANDRADINA,-22.0785,-53.4659,meterological_station,inmet,S713
584,869,PEDRO GOMES,-18.0727,-54.5488,meterological_station,inmet,S714
585,870,RIBAS DO RIO PARDO,-20.4667,-53.7630,meterological_station,inmet,S715
586,871,SANTA RITA DO PARDO,-21.3059,-52.8204,meterological_station,inmet,S716


In [39]:
url = f"{api}/estacao/{start_date}/{end_date}/{stations.iloc[0].url}"
url

'https://apitempo.inmet.gov.br/estacao/2021-10-16/2021-10-17/A001'

In [40]:
response = requests.get(url).json()

In [41]:
df = pd.DataFrame(response)
df = df[['PRE_INS', 'PTO_INS', 'VEN_DIR', 'DT_MEDICAO', 'CHUVA', 'VEN_VEL','VEN_RAJ', 'TEM_INS',
         'UMD_INS', 'HR_MEDICAO']]
df.columns = ['pres', 'dewp', 'wdir', 'date', 'prec', 'wspd', 'gust', 'atmp', 'humi', 'hour']
df['date_time'] = pd.to_datetime(df['date'] + df['hour'], format='%Y-%m-%d%H%M')
df.drop(columns=['date', 'hour'], inplace=True)
df = df[df.date_time <datetime.utcnow()]

In [42]:
df.tail(40)

,pres,dewp,wdir,prec,wspd,gust,atmp,humi,date_time
0,886.7,16.4,312,0,1.5,3.4,22.2,70,2021-10-16 00:00:00
1,888.1,15.9,339,0,1.9,3.1,22.4,67,2021-10-16 01:00:00
2,888.3,15.7,331,0,1.8,4.5,22.2,67,2021-10-16 02:00:00
3,887.5,15.8,344,0,2.3,4.4,22.5,66,2021-10-16 03:00:00
4,887.1,16.2,336,0,2.5,5,21.5,72,2021-10-16 04:00:00
5,886.9,16.3,311,0,1.3,5,20.1,79,2021-10-16 05:00:00
6,886.2,16.3,320,0,1.3,2.7,20,79,2021-10-16 06:00:00
7,886.6,16.6,312,0,1.4,2.1,20.6,78,2021-10-16 07:00:00
8,887.2,17,39,0,0.2,2.5,20.3,81,2021-10-16 08:00:00
9,888,17.5,45,0,0.5,0.8,19.9,86,2021-10-16 09:00:00


In [32]:
df.date_time<datetime.utcnow()

0      True
1      True
2      True
3      True
4      True
      ...  
67    False
68    False
69    False
70    False
71    False
Name: date_time, Length: 72, dtype: bool

0     2021-10-160000
1     2021-10-160100
2     2021-10-160200
3     2021-10-160300
4     2021-10-160400
           ...      
67    2021-10-181900
68    2021-10-182000
69    2021-10-182100
70    2021-10-182200
71    2021-10-182300
Length: 72, dtype: object

In [21]:
df

,pres,dewp,wdir,date,prec,wspd,gust,atmp,humi,hour
0,885.3,18.9,79,2021-10-14,0,0.8,3.4,21.9,83,0000
1,885.5,18.7,89,2021-10-14,0,2.6,5.2,22,81,0100
2,885.2,18.6,45,2021-10-14,0,1.4,5.2,21.8,82,0200
3,884.9,18.5,36,2021-10-14,0,0.8,3.7,21.1,85,0300
4,884.5,18.6,3,2021-10-14,0,1.7,2.8,21.1,86,0400
...,...,...,...,...,...,...,...,...,...,...
115,None,None,None,2021-10-18,None,None,None,None,None,1900
116,None,None,None,2021-10-18,None,None,None,None,None,2000
117,None,None,None,2021-10-18,None,None,None,None,None,2100
118,None,None,None,2021-10-18,None,None,None,None,None,2200


In [12]:
p = Pnboia()

In [13]:
p.get()

,date_time,lat,lon,rh,pres,atmp,dewpt,wspd,wdir,gust,arad,sst,swvht,mxwvht,tp,wvdir
0,2021-10-14 00:00:00,-25.5146,-42.7257,80.5,1018.1,22.0,18.5,0.45,183,14.74,0.03,21.96,3.30,4.77,8.7,106
1,2021-10-14 01:00:00,-25.5144,-42.7257,80.7,1017.5,21.9,18.6,1.25,123,15.42,0.02,21.94,3.36,5.32,8.3,97
2,2021-10-14 03:00:00,-25.5135,-42.7252,80.7,1016.3,21.9,18.7,0.34,242,14.74,0.02,21.89,3.22,5.27,9.1,95
3,2021-10-14 04:00:00,-25.5129,-42.7251,80.3,1015.8,21.9,18.5,0.68,189,14.74,0.04,21.87,3.25,4.93,8.7,88
4,2021-10-14 05:00:00,-25.5122,-42.7248,81.9,1015.3,21.8,18.4,0.79,191,15.87,0.02,21.86,3.14,4.91,9.1,93
5,2021-10-14 06:00:00,-25.5113,-42.7246,81.6,1014.6,21.8,18.2,0.79,96,13.94,0.04,21.85,3.15,5.11,9.1,85
6,2021-10-14 07:00:00,-25.5102,-42.7245,82.5,1014.6,21.7,18.6,0.91,182,13.72,0.00,21.82,2.95,4.30,8.3,95
7,2021-10-14 08:00:00,-25.5091,-42.7244,85.2,1014.8,21.6,19.1,1.02,169,11.68,0.03,21.80,3.17,6.13,8.7,89
8,2021-10-14 09:00:00,-25.5082,-42.7245,84.3,1015.6,21.7,19.1,0.57,183,11.22,0.08,21.78,3.14,4.52,9.1,81
9,2021-10-14 10:00:00,-25.5079,-42.7246,84.7,1015.9,21.8,19.3,0.91,178,12.24,0.08,21.77,3.57,5.06,8.7,80


In [4]:
bd = GetData()

In [4]:
bd.get(table='stations', institution=['=', 'simcosta'], data_type=['=', 'buoy'])

,id,name,lat,lon,data_type,institution,url
0,897,RJ3,-22.9831,-43.1745,buoy,simcosta,3
1,898,RS2,-32.1346,-52098.0000,buoy,simcosta,6
2,899,RS5,-32296.0000,-52.0239,buoy,simcosta,10
3,900,RJ4,-22.9715,-43.1522,buoy,simcosta,12
4,901,RS3,-32195.0000,-52.0794,buoy,simcosta,13
5,902,RS4,-32.2454,-52.0954,buoy,simcosta,14
6,903,BA1,-12.9895,-37.4582,buoy,simcosta,15


In [8]:
df = bd.get(table='data_stations', station_id=['in', [902]])

In [9]:
df.shape

(94, 22)

In [7]:
bd.post(table='data_stations', df=df)

DELETE FROM data_stations WHERE true AND station_id in (902) AND date_time >= '2021-09-06 00:21:00'


In [3]:
import time
import datetime
import operator
import urllib.request, json

import numpy as np
from oceanoobsbrasil.buoys.simcosta import Simcosta
from oceanoobsbrasil.tides.simcosta import SimcostaTide

In [4]:
s = SimcostaTide()

In [5]:
s.get()

Nao ha dados para essa boia
Nao ha dados para essa boia
Nao ha dados para essa boia


KeyboardInterrupt: 

In [52]:
s = Simcosta()

In [53]:
s.get()

id                  897
name                RJ3
lat            -22.9831
lon            -43.1745
data_type          buoy
institution    simcosta
url                   3
Name: 0, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
Empty D

    YEAR MONTH DAY HOUR MINUTE SECOND Avg_Air_Press Avg_Air_Tmp    Hmt  \
0   2021     9   8    3     21     00       1018.53        None  45.20   
1   2021     9   8    3     51     00       1018.39        None  44.30   
2   2021     9   8    4     51     00       1017.51        None  44.00   
3   2021     9   8    5     21     00       1017.30        None  44.60   
4   2021     9   8    5     51     00       1016.99        None  43.80   
5   2021     9   8    6     51     00       1015.98        None  43.70   
6   2021     9   8    7     21     00       1015.71        None  44.60   
7   2021     9   8    7     51     00       1015.55        None  43.60   
8   2021     9   8    8     21     00       1015.67        None  44.50   
9   2021     9   8    9     21     00       1016.01        None  44.50   
10  2021     9   8    9     51     00       1016.16        None  43.70   
11  2021     9   8   10     21     00       1016.07        None  45.40   
12  2021     9   8   10     51     00 

ok2
id                  903
name                BA1
lat            -12.9895
lon            -37.4582
data_type          buoy
institution    simcosta
url                  15
Name: 6, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=15&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=15&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
 

In [40]:
                columns = ['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND','Hmt',
                    'Avg_Wnd_Dir','M_Decl','Avg_W_Tmp1','Avg_Sal','Avg_Spre_N',
                    'Avg_Wv_Dir','Avg_Cel1_Mag','Avg_Cel1_Dir','Avg_Cel1_Dir_N',
                    'Avg_Turb','Avg_Chl','Avg_DO','ZCN','HM0','TAvg','Tp5',
                    'T10','HAvg','Tsig','CDOM','H10','Avg_Sol_Rad']



In [43]:
s.result.drop(columns=columns, inplace=True)

In [44]:
s.result.columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

In [45]:
s.result = s.result.replace(to_replace =['None', 'NULL', ' ', ''],
                                        value =np.nan)

In [46]:
s.result['station_id'] = 898

In [ ]:
s.